# Shut the Box

## Step 0: Utils for Jupyter Notebooks

### Add button to Show/Hide code (to focus on outputs)

In [17]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Show Code"></form>''')

### Add magic command to store python variables in csv file

In [45]:
# credit to sirex, https://github.com/ipython/ipython/issues/6701
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

## Step 1: Generate models

In [46]:
# Edit this cell to define parameters for your model...

boards = 6 # Number of boards to use
sides = 6 # Number of sides per die
die = 1 # Number of die to roll each round

max_score = boards * (boards+1) // 2

In [47]:
%%writetemplate ShutTheBox/params/stb_params_high.csv
b, {boards}
d, {sides}
ndie, {die}
strategy, 1

In [48]:
%%writetemplate ShutTheBox/params/stb_params_low.csv
b, {boards}
d, {sides}
ndie, {die}
strategy, 2

In [49]:
%%writetemplate ShutTheBox/params/stb_nondet_params.csv
b, {boards}
d, {sides}
ndie, {die}
strategy, 0

In [50]:
!mkdir test

A subdirectory or file test already exists.


In [51]:
model_spec = f"stb{boards}_{die}d{sides}"

!mkdir "ShutTheBox/models/$model_spec"
output_model_high = f"ShutTheBox/models/{model_spec}/{model_spec}_high.prism"

!python utils/pyprism.py ShutTheBox/stb.pyprism $output_model_high ShutTheBox/params/stb_params_high.csv

A subdirectory or file ShutTheBox/models/stb6_1d6 already exists.


Model generated successfully at ShutTheBox\models\stb6_1d6\stb6_1d6_high.prism


In [52]:
output_model_low = f"ShutTheBox/models/{model_spec}/{model_spec}_low.prism"

!python utils/pyprism.py ShutTheBox/stb.pyprism $output_model_low ShutTheBox/params/stb_params_low.csv

Model generated successfully at ShutTheBox\models\stb6_1d6\stb6_1d6_low.prism


In [53]:
output_model_nondet = f"ShutTheBox/models/{model_spec}/{model_spec}_nondet.prism"

!python utils/pyprism.py ShutTheBox/stb.pyprism $output_model_nondet ShutTheBox/params/stb_nondet_params.csv

Model generated successfully at ShutTheBox\models\stb6_1d6\stb6_1d6_nondet.prism


In [54]:
output_props = f"ShutTheBox/props/{model_spec}.props"

!python utils/pyprism.py ShutTheBox/stbprops.pyprism $output_props ShutTheBox/params/stb_params_high.csv

Model generated successfully at ShutTheBox\props\stb6_1d6.props


## Step 2: Check properties, run experiments, export data

### Does the game eventually always terminate?

We expect to see a value of approximately 1 here (it won't be exact because CSGs only support methods based on numerical convergence, so it may be slightly under or even over 1!

In [55]:
!prism $output_model_high $output_props -prop 1 | grep -E "(Model checking)|(Result)|(sec)|(Error)"

Model checking: Pmax=? [ F game_over ]
Reachability (BFS): 12 iterations in 0.00 seconds (average 0.000000, setup 0.00)
Time for model construction: 0.049 seconds.
Prob0A: 7 iterations in 0.00 seconds (average 0.000143, setup 0.00)
Prob1E: 8 iterations in 0.00 seconds (average 0.000125, setup 0.00)
Time for model checking: 0.003 seconds.
Result: 1.0 (value in the initial state)


### Probability of obtaining each score 

In [56]:
data_location = f"ShutTheBox/data/{model_spec}"

!mkdir "$data_location"

prob_score_high_file = f"{data_location}/{model_spec}_prob_score_high.csv"

!prism $output_model_high $output_props -prop 2 -const k=1:{max_score} -exportresults {prob_score_high_file}:csv | grep -E "(Model checking)|(Result)|(Property constants)|(Error)"

A subdirectory or file ShutTheBox/data/stb6_1d6 already exists.


Model checking: Pmin=? [ F game_over&score=k ]
Property constants: k=1
Result: 0.027777777777777776 (value in the initial state)
Model checking: Pmin=? [ F game_over&score=k ]
Property constants: k=2
Result: 0.027777777777777776 (value in the initial state)
Model checking: Pmin=? [ F game_over&score=k ]
Property constants: k=3
Result: 0.018518518518518517 (value in the initial state)
Model checking: Pmin=? [ F game_over&score=k ]
Property constants: k=4
Result: 0.018518518518518517 (value in the initial state)
Model checking: Pmin=? [ F game_over&score=k ]
Property constants: k=5
Result: 0.037037037037037035 (value in the initial state)
Model checking: Pmin=? [ F game_over&score=k ]
Property constants: k=6
Result: 0.046296296296296294 (value in the initial state)
Model checking: Pmin=? [ F game_over&score=k ]
Property constants: k=7
Result: 0.041666666666666664 (value in the initial state)
Model checking: Pmin=? [ F game_over&score=k ]
Property constants: k=8
Result: 0.0509259259259259

In [57]:
prob_score_low_file = f"{data_location}/{model_spec}_prob_score_low.csv"

!prism $output_model_low $output_props -prop 2 -const k=1:{max_score} -exportresults {prob_score_low_file}:csv | grep -E "(Model checking)|(Result)|(Property constants)|(sec)|(Error)"

Model checking: Pmin=? [ F game_over&score=k ]
Property constants: k=1
Reachability (BFS): 12 iterations in 0.00 seconds (average 0.000083, setup 0.00)
Time for model construction: 0.049 seconds.
Prob0E: 6 iterations in 0.00 seconds (average 0.000000, setup 0.00)
Prob1A: 3 iterations in 0.00 seconds (average 0.000333, setup 0.00)
Iterative method: 4 iterations in 0.00 seconds (average 0.000000, setup 0.00)
Time for model checking: 0.005 seconds.
Result: 0.027777777777777776 (value in the initial state)
Model checking: Pmin=? [ F game_over&score=k ]
Property constants: k=2
Prob0E: 6 iterations in 0.00 seconds (average 0.000000, setup 0.00)
Prob1A: 3 iterations in 0.00 seconds (average 0.000000, setup 0.00)
Iterative method: 4 iterations in 0.00 seconds (average 0.000000, setup 0.00)
Time for model checking: 0.002 seconds.
Result: 0.027777777777777776 (value in the initial state)
Model checking: Pmin=? [ F game_over&score=k ]
Property constants: k=3
Prob0E: 7 iterations in 0.00 seconds (

In [58]:
prob_score_nondet_min_file = f"{data_location}/{model_spec}_prob_score_nondet_min.csv"

!prism $output_model_nondet $output_props -prop 2 -const k=1:{max_score} -exportresults {prob_score_nondet_min_file}:csv | grep -E "(Model checking)|(Result)|(Property constants)|(Error)"

Model checking: Pmin=? [ F game_over&score=k ]
Property constants: k=1
Result: 0.027777777777777776 (value in the initial state)
Model checking: Pmin=? [ F game_over&score=k ]
Property constants: k=2
Result: 0.027777777777777776 (value in the initial state)
Model checking: Pmin=? [ F game_over&score=k ]
Property constants: k=3
Result: 0.018518518518518517 (value in the initial state)
Model checking: Pmin=? [ F game_over&score=k ]
Property constants: k=4
Result: 0.009259259259259259 (value in the initial state)
Model checking: Pmin=? [ F game_over&score=k ]
Property constants: k=5
Result: 0.018518518518518517 (value in the initial state)
Model checking: Pmin=? [ F game_over&score=k ]
Property constants: k=6
Result: 0.037037037037037035 (value in the initial state)
Model checking: Pmin=? [ F game_over&score=k ]
Property constants: k=7
Result: 0.020833333333333332 (value in the initial state)
Model checking: Pmin=? [ F game_over&score=k ]
Property constants: k=8
Result: 0.0324074074074074

In [59]:
prob_score_nondet_max_file = f"{data_location}/{model_spec}_prob_score_nondet_max.csv"

!prism $output_model_nondet $output_props -prop 3 -const k=1:{max_score} -exportresults {prob_score_nondet_max_file}:csv | grep -E "(Model checking)|(Result)|(Property constants)|(Error)"

Model checking: Pmax=? [ F game_over&score=k ]
Property constants: k=1
Result: 0.027777777777777776 (value in the initial state)
Model checking: Pmax=? [ F game_over&score=k ]
Property constants: k=2
Result: 0.027777777777777776 (value in the initial state)
Model checking: Pmax=? [ F game_over&score=k ]
Property constants: k=3
Result: 0.07407407407407407 (value in the initial state)
Model checking: Pmax=? [ F game_over&score=k ]
Property constants: k=4
Result: 0.07407407407407407 (value in the initial state)
Model checking: Pmax=? [ F game_over&score=k ]
Property constants: k=5
Result: 0.09259259259259259 (value in the initial state)
Model checking: Pmax=? [ F game_over&score=k ]
Property constants: k=6
Result: 0.16666666666666669 (value in the initial state)
Model checking: Pmax=? [ F game_over&score=k ]
Property constants: k=7
Result: 0.09722222222222224 (value in the initial state)
Model checking: Pmax=? [ F game_over&score=k ]
Property constants: k=8
Result: 0.09722222222222221 (va

### Expected number of die rolls

In [60]:
print("High board strategy:")
!prism $output_model_high $output_props -prop 9 | grep -E "(Model checking)|(Result)|(Error)"
print("Low board strategy:")
!prism $output_model_low $output_props -prop 9 | grep -E "(Model checking)|(Result)|(Error)"

High board strategy:
Model checking: R{"no_rolls"}max=? [ F game_over ]
Result: 4.313786008230452 (value in the initial state)
Low board strategy:
Model checking: R{"no_rolls"}max=? [ F game_over ]
Result: 3.214077503429355 (value in the initial state)


### Probability of obtaining a score given a particular board is covered

In [61]:
for covered_board in range(1, boards+1):
    !prism $output_model_high $output_props -prop {10+covered_board} -const k=1:{max_score} -exportresults ShutTheBox/data/{model_spec}/{model_spec}_score_with_covered_{covered_board}.csv:csv | grep -E "(Model checking)|(Result)|(Property constant)|(Exporting results)|(Error)"

Model checking: Pmin=? [ F game_over&b1=1&score=k ]
Property constants: k=1
Result: 0.027777777777777776 (value in the initial state)
Model checking: Pmin=? [ F game_over&b1=1&score=k ]
Property constants: k=2
Result: 0.0 (value in the initial state)
Model checking: Pmin=? [ F game_over&b1=1&score=k ]
Property constants: k=3
Result: 0.018518518518518517 (value in the initial state)
Model checking: Pmin=? [ F game_over&b1=1&score=k ]
Property constants: k=4
Result: 0.018518518518518517 (value in the initial state)
Model checking: Pmin=? [ F game_over&b1=1&score=k ]
Property constants: k=5
Result: 0.018518518518518517 (value in the initial state)
Model checking: Pmin=? [ F game_over&b1=1&score=k ]
Property constants: k=6
Result: 0.037037037037037035 (value in the initial state)
Model checking: Pmin=? [ F game_over&b1=1&score=k ]
Property constants: k=7
Result: 0.023148148148148147 (value in the initial state)
Model checking: Pmin=? [ F game_over&b1=1&score=k ]
Property constants: k=8
Res

Model checking: Pmin=? [ F game_over&b4=1&score=k ]
Property constants: k=1
Result: 0.0 (value in the initial state)
Model checking: Pmin=? [ F game_over&b4=1&score=k ]
Property constants: k=2
Result: 0.0 (value in the initial state)
Model checking: Pmin=? [ F game_over&b4=1&score=k ]
Property constants: k=3
Result: 0.0 (value in the initial state)
Model checking: Pmin=? [ F game_over&b4=1&score=k ]
Property constants: k=4
Result: 0.0 (value in the initial state)
Model checking: Pmin=? [ F game_over&b4=1&score=k ]
Property constants: k=5
Result: 0.018518518518518517 (value in the initial state)
Model checking: Pmin=? [ F game_over&b4=1&score=k ]
Property constants: k=6
Result: 0.009259259259259259 (value in the initial state)
Model checking: Pmin=? [ F game_over&b4=1&score=k ]
Property constants: k=7
Result: 0.023148148148148147 (value in the initial state)
Model checking: Pmin=? [ F game_over&b4=1&score=k ]
Property constants: k=8
Result: 0.027777777777777776 (value in the initial sta

### Cumulative probability of a score

In [62]:
!prism $output_model_high $output_props -prop 6 -const k=1:{max_score} -exportresults ShutTheBox/data/{model_spec}/{model_spec}_cum_prob_high.csv:csv | grep -E "(Model checking)|(Result)|(Property constant)|(Exporting results)|(Error)"

Model checking: Pmax=? [ F game_over&score<=k ]
Property constants: k=1
Result: 0.027777777777777776 (value in the initial state)
Model checking: Pmax=? [ F game_over&score<=k ]
Property constants: k=2
Result: 0.05555555555555555 (value in the initial state)
Model checking: Pmax=? [ F game_over&score<=k ]
Property constants: k=3
Result: 0.07407407407407407 (value in the initial state)
Model checking: Pmax=? [ F game_over&score<=k ]
Property constants: k=4
Result: 0.09259259259259259 (value in the initial state)
Model checking: Pmax=? [ F game_over&score<=k ]
Property constants: k=5
Result: 0.12962962962962965 (value in the initial state)
Model checking: Pmax=? [ F game_over&score<=k ]
Property constants: k=6
Result: 0.1759259259259259 (value in the initial state)
Model checking: Pmax=? [ F game_over&score<=k ]
Property constants: k=7
Result: 0.2175925925925926 (value in the initial state)
Model checking: Pmax=? [ F game_over&score<=k ]
Property constants: k=8
Result: 0.2685185185185184

In [63]:
!prism $output_model_low $output_props -prop 6 -const k=1:{max_score} -exportresults ShutTheBox/data/{model_spec}/{model_spec}_cum_prob_low.csv:csv | grep -E "(Model checking)|(Result)|(Property constant)|(Exporting results)|(Error)"

Model checking: Pmax=? [ F game_over&score<=k ]
Property constants: k=1
Result: 0.027777777777777776 (value in the initial state)
Model checking: Pmax=? [ F game_over&score<=k ]
Property constants: k=2
Result: 0.05555555555555555 (value in the initial state)
Model checking: Pmax=? [ F game_over&score<=k ]
Property constants: k=3
Result: 0.12962962962962962 (value in the initial state)
Model checking: Pmax=? [ F game_over&score<=k ]
Property constants: k=4
Result: 0.19444444444444445 (value in the initial state)
Model checking: Pmax=? [ F game_over&score<=k ]
Property constants: k=5
Result: 0.2685185185185185 (value in the initial state)
Model checking: Pmax=? [ F game_over&score<=k ]
Property constants: k=6
Result: 0.4305555555555554 (value in the initial state)
Model checking: Pmax=? [ F game_over&score<=k ]
Property constants: k=7
Result: 0.4791666666666666 (value in the initial state)
Model checking: Pmax=? [ F game_over&score<=k ]
Property constants: k=8
Result: 0.5300925925925926 

In [64]:
!prism $output_model_nondet $output_props -prop 5 -const k=1:{max_score} -exportresults ShutTheBox/data/{model_spec}/{model_spec}_cum_prob_min.csv:csv | grep -E "(Model checking)|(Result)|(Property constant)|(Exporting results)|(Error)"

Model checking: Pmin=? [ F game_over&score<=k ]
Property constants: k=1
Result: 0.027777777777777776 (value in the initial state)
Model checking: Pmin=? [ F game_over&score<=k ]
Property constants: k=2
Result: 0.05555555555555555 (value in the initial state)
Model checking: Pmin=? [ F game_over&score<=k ]
Property constants: k=3
Result: 0.07407407407407407 (value in the initial state)
Model checking: Pmin=? [ F game_over&score<=k ]
Property constants: k=4
Result: 0.09259259259259259 (value in the initial state)
Model checking: Pmin=? [ F game_over&score<=k ]
Property constants: k=5
Result: 0.12962962962962965 (value in the initial state)
Model checking: Pmin=? [ F game_over&score<=k ]
Property constants: k=6
Result: 0.1759259259259259 (value in the initial state)
Model checking: Pmin=? [ F game_over&score<=k ]
Property constants: k=7
Result: 0.2175925925925926 (value in the initial state)
Model checking: Pmin=? [ F game_over&score<=k ]
Property constants: k=8
Result: 0.2685185185185184

In [65]:
cum_prob_max_file = "ShutTheBox/data/cum_prob_max.csv"

!prism $output_model_nondet $output_props -prop 6 -const k=1:{max_score} -exportresults ShutTheBox/data/{model_spec}/{model_spec}_cum_prob_max.csv:csv | grep -E "(Model checking)|(Result)|(Property constant)|(Exporting results)|(Error)"

Model checking: Pmax=? [ F game_over&score<=k ]
Property constants: k=1
Result: 0.027777777777777776 (value in the initial state)
Model checking: Pmax=? [ F game_over&score<=k ]
Property constants: k=2
Result: 0.05555555555555555 (value in the initial state)
Model checking: Pmax=? [ F game_over&score<=k ]
Property constants: k=3
Result: 0.12962962962962962 (value in the initial state)
Model checking: Pmax=? [ F game_over&score<=k ]
Property constants: k=4
Result: 0.19444444444444445 (value in the initial state)
Model checking: Pmax=? [ F game_over&score<=k ]
Property constants: k=5
Result: 0.2685185185185185 (value in the initial state)
Model checking: Pmax=? [ F game_over&score<=k ]
Property constants: k=6
Result: 0.4305555555555554 (value in the initial state)
Model checking: Pmax=? [ F game_over&score<=k ]
Property constants: k=7
Result: 0.4791666666666666 (value in the initial state)
Model checking: Pmax=? [ F game_over&score<=k ]
Property constants: k=8
Result: 0.5439814814814814 

# Optimal Strategy for highest expected score

In [77]:
!prism $output_model_nondet $output_props -prop 8 -exportresults ShutTheBox/data/{model_spec}/{model_spec}_exp_score_min.csv:csv | grep -E "(Model checking)|(Result)|(Property constant)|(Exporting results)|(Error)"

Model checking: R{"total_boards"}max=? [ F game_over ]
Result: 12.148662551440324 (value in the initial state)
Exporting results to file "ShutTheBox/data/stb6_1d6/stb6_1d6_exp_score_min.csv"...


In [78]:
!prism $output_model_nondet $output_props -prop 8  -s -exportadvmdp ShutTheBox/data/{model_spec}/{model_spec}_exp_score_min.tra -exportstates ShutTheBox/data/{model_spec}/{model_spec}_exp_score_min.sta -exportlabels ShutTheBox/data/{model_spec}/{model_spec}_exp_score_min.lab -exportstaterewards ShutTheBox/data/{model_spec}/state_rewards.txt -exporttransrewards ShutTheBox/data/{model_spec}/{model_spec}_trans_rewards.txt

PRISM-games

Version: 3.0 (based on PRISM 4.6)
Date: Wed Mar 24 11:23:31 GMT 2021
Hostname: DESKTOP-LDYER
Memory limits: cudd=1g, java(heap)=910.5m
Command line: prism-games ShutTheBox/models/stb6_1d6/stb6_1d6_nondet.prism ShutTheBox/props/stb6_1d6.props -prop 8 -s -exportadvmdp ShutTheBox/data/stb6_1d6/stb6_1d6_exp_score_min.tra -exportstates ShutTheBox/data/stb6_1d6/stb6_1d6_exp_score_min.sta -exportlabels ShutTheBox/data/stb6_1d6/stb6_1d6_exp_score_min.lab -exportstaterewards ShutTheBox/data/stb6_1d6/state_rewards.txt -exporttransrewards ShutTheBox/data/stb6_1d6/stb6_1d6_trans_rewards.txt

Parsing model file "ShutTheBox/models/stb6_1d6/stb6_1d6_nondet.prism"...

Type:        MDP
Modules:     player1 board1 board2 board3 board4 board5 board6 
Variables:   state die b1 b2 b3 b4 b5 b6 

Parsing properties file "ShutTheBox/props/stb6_1d6.props"...

16 properties:
(1) Pmax=? [ F game_over ]
(2) Pmin=? [ F game_over&score=k ]
(3) Pmax=? [ F game_over&score=k ]
(4) Pmax=? [ F game_over&sco

In [79]:
!prism -mdp -importtrans ShutTheBox/data/{model_spec}/{model_spec}_exp_score_min.tra -importstates ShutTheBox/data/{model_spec}/{model_spec}_exp_score_min.sta -importlabels ShutTheBox/data/{model_spec}/{model_spec}_exp_score_min.lab -exporttrans test.tra -exportstates test.sta -exporttransdotstates test.dot -fixdl

PRISM-games

Version: 3.0 (based on PRISM 4.6)
Date: Wed Mar 24 11:23:40 GMT 2021
Hostname: DESKTOP-LDYER
Memory limits: cudd=1g, java(heap)=910.5m
Command line: prism-games -mdp -importtrans ShutTheBox/data/stb6_1d6/stb6_1d6_exp_score_min.tra -importstates ShutTheBox/data/stb6_1d6/stb6_1d6_exp_score_min.sta -importlabels ShutTheBox/data/stb6_1d6/stb6_1d6_exp_score_min.lab -exporttrans test.tra -exportstates test.sta -exporttransdotstates test.dot -fixdl

Importing model from "ShutTheBox/data/stb6_1d6/stb6_1d6_exp_score_min.tra", "ShutTheBox/data/stb6_1d6/stb6_1d6_exp_score_min.sta", "ShutTheBox/data/stb6_1d6/stb6_1d6_exp_score_min.lab"...
Using specified model type: MDP

Building model...

Computing reachable states...

Reachability (BFS): 12 iterations in 0.00 seconds (average 0.000083, setup 0.00)

Time for model construction: 0.099 seconds.


Type:        MDP
States:      871 (1 initial)
Transitions: 1836
Choices:     871

Transition matrix: 356 nodes (3 terminal), 1836 minterms, v

## Try for highboard

In [80]:
!prism $output_model_high $output_props -prop 7  -s -exportadvmdp ShutTheBox/data/{model_spec}/{model_spec}_exp_score_highboard.tra -exportstates ShutTheBox/data/{model_spec}/{model_spec}_exp_score_highboard.sta -exportlabels ShutTheBox/data/{model_spec}/{model_spec}_exp_score_highboard.lab -exportstaterewards ShutTheBox/data/{model_spec}/state_rewards_high.txt -exporttransrewards ShutTheBox/data/{model_spec}/{model_spec}_trans_rewards_high.txt

PRISM-games

Version: 3.0 (based on PRISM 4.6)
Date: Wed Mar 24 11:25:07 GMT 2021
Hostname: DESKTOP-LDYER
Memory limits: cudd=1g, java(heap)=910.5m
Command line: prism-games ShutTheBox/models/stb6_1d6/stb6_1d6_high.prism ShutTheBox/props/stb6_1d6.props -prop 7 -s -exportadvmdp ShutTheBox/data/stb6_1d6/stb6_1d6_exp_score_highboard.tra -exportstates ShutTheBox/data/stb6_1d6/stb6_1d6_exp_score_highboard.sta -exportlabels ShutTheBox/data/stb6_1d6/stb6_1d6_exp_score_highboard.lab -exportstaterewards ShutTheBox/data/stb6_1d6/state_rewards_high.txt -exporttransrewards ShutTheBox/data/stb6_1d6/stb6_1d6_trans_rewards_high.txt

Parsing model file "ShutTheBox/models/stb6_1d6/stb6_1d6_high.prism"...

Type:        MDP
Modules:     player1 board1 board2 board3 board4 board5 board6 
Variables:   state die b1 b2 b3 b4 b5 b6 

Parsing properties file "ShutTheBox/props/stb6_1d6.props"...

16 properties:
(1) Pmax=? [ F game_over ]
(2) Pmin=? [ F game_over&score=k ]
(3) Pmax=? [ F game_over&score=k ]
(4) 

In [73]:
!prism -mdp -importtrans ShutTheBox/data/{model_spec}/{model_spec}_exp_score_highboard.tra -importstates ShutTheBox/data/{model_spec}/{model_spec}_exp_score_highboard.sta -importlabels ShutTheBox/data/{model_spec}/{model_spec}_exp_score_highboard.lab -exporttrans test_highboard.tra -exportstates test_highboard.sta -exporttransdotstates test_highboard.dot -fixdl

PRISM-games

Version: 3.0 (based on PRISM 4.6)
Date: Wed Mar 24 10:32:52 GMT 2021
Hostname: DESKTOP-LDYER
Memory limits: cudd=1g, java(heap)=910.5m
Command line: prism-games -mdp -importtrans ShutTheBox/data/stb6_1d6/stb6_1d6_exp_score_highboard.tra -importstates ShutTheBox/data/stb6_1d6/stb6_1d6_exp_score_highboard.sta -importlabels ShutTheBox/data/stb6_1d6/stb6_1d6_exp_score_highboard.lab -exporttrans test_highboard.tra -exportstates test_highboard.sta -exporttransdotstates test_highboard.dot -fixdl

Importing model from "ShutTheBox/data/stb6_1d6/stb6_1d6_exp_score_highboard.tra", "ShutTheBox/data/stb6_1d6/stb6_1d6_exp_score_highboard.sta", "ShutTheBox/data/stb6_1d6/stb6_1d6_exp_score_highboard.lab"...
Using specified model type: MDP

Building model...

Computing reachable states...

Reachability (BFS): 12 iterations in 0.00 seconds (average 0.000083, setup 0.00)

Time for model construction: 0.099 seconds.


Type:        MDP
States:      871 (1 initial)
Transitions: 1836
Choices:   

In [69]:
!prism $output_model_nondet $output_props -prop 8 -exportresults ShutTheBox/data/{model_spec}/{model_spec}_exp_score_max.csv:csv -s | grep -E "(Model checking)|(Result)|(Property constant)|(Exporting results)|(Error)"

Model checking: R{"total_boards"}max=? [ F game_over ]
Result: 12.148662551440324 (value in the initial state)
Exporting results to file "ShutTheBox/data/stb6_1d6/stb6_1d6_exp_score_max.csv"...


In [70]:
!prism $output_model_low $output_props -prop 8 -exportresults ShutTheBox/data/{model_spec}/{model_spec}_exp_score_low.csv:csv | grep -E "(Model checking)|(Result)|(Property constant)|(Exporting results)|(Error)"

Model checking: R{"total_boards"}max=? [ F game_over ]
Result: 8.753150720164609 (value in the initial state)
Exporting results to file "ShutTheBox/data/stb6_1d6/stb6_1d6_exp_score_low.csv"...


In [71]:
!prism $output_model_high $output_props -prop 8 -exportresults ShutTheBox/data/{model_spec}/{model_spec}_exp_score_high.csv:csv | grep -E "(Model checking)|(Result)|(Property constant)|(Exporting results)|(Error)"

Model checking: R{"total_boards"}max=? [ F game_over ]
Result: 12.148662551440324 (value in the initial state)
Exporting results to file "ShutTheBox/data/stb6_1d6/stb6_1d6_exp_score_high.csv"...
